In [ ]:
import tifffile
import pandas

from griottes.analyse import cell_property_extraction
from griottes.graphmaker import graph_generation_func
from griottes.graphplotter import graph_plot
import griottes

## Figure 3: Network representation of the spheroid

From the image we can extract the single cell information. For the analysis, it is necessary that the channels be structured as $Z \times X \times Y \times C$, where $C$ is the fluorescence channels in a multi-channel image.

The file `total_image.tiff` can be downloaded from XXX.

In [ ]:
total_image = tifffile.imread('total_image.tiff')
total_image = total_image[110:400, ...]

In [ ]:
prop, vor = cell_property_extraction.get_cell_properties(
    total_image.astype(int),
    mask_channel = 3,
    analyze_fluo_channels = True,
    fluo_channel_analysis_method = 'local_voronoi',
    cell_geometry_properties = True,
    radius = 35,
    labeled_voronoi_tesselation = False,
    percentile = 95,
    min_area = 4000
    )

From the extracted positions of the cell centers, it is possible to generate a network representation of the 3D spheroid. For practical purposes, we have saved the dataframe containing the previously extracted information which is re-used below.

In [ ]:
prop = pandas.read_csv('prop.csv')

In [ ]:
prop['cell_type'] = (prop.mean_intensity_1 > prop.mean_intensity_1.median()).astype(int)
prop.index = np.arange(len(prop))

legend_list = ['CD146-', 'CD146+']
prop['legend'] = [legend_list[prop.loc[i, 'cell_type']] for i in range(len(prop))]

color_list = [plt.cm.Set3(i) for i in range(2)]
prop['color'] = [color_list[prop.loc[i, 'cell_type']] for i in range(len(prop))]


In [ ]:
descriptors = ['x', 'y', 'legend', 'color', 'cell_type']

G = graph_generation_func.generate_delaunay_graph(prop, 
                                                 descriptors = descriptors, 
                                                 dCells = 100,
                                                 flat_image = True)

In [ ]:
graph_plot.network_plot_3D(G, 
                figsize = (7, 7),
                alpha_line=0.6,
                scatterpoint_size=10,
                legend=True,
                legend_fontsize = 18,
                theta = -0,
                psi = -0,
                xlim = (prop.x.min() - 5, prop.x.max() + 5),
                ylim = (prop.y.min() - 5, prop.y.max() + 5),
                zlim = (prop.z.min() - 5, prop.z.max() + 5))